# Дипломный проект по SQL

## Описание проекта
Наша компания купила купила крупный сервис для чтения книг по подписке. Сейчас важно сформулировать ценностное предложение для нового продукта.

## Задача исследования  
Проанализировать базу данных с информацией о книгах, издательствах, авторах, а также пользовательскими обзорами книг, чтобы предоставить основные тезисы для уникального торгового предложения.

## Цели исследования

В результате анализа необходимо:

1. Изучить структуру базы данных — определить, какие таблицы содержат информацию о книгах, авторах, издательствах, пользовательских оценках и обзорах.

2. Оценить активность книгоиздания — посчитать, сколько книг вышло после 1 января 2000 года, чтобы понять, насколько активно обновляется контент.

3. Изучить вовлечённость пользователей — рассчитать количество обзоров и средние оценки для каждой книги.

4. Выявить ключевого издателя — определить издательство, выпустившее наибольшее число книг объёмом более 50 страниц.

5. Найти автора с самой высокой средней оценкой — но учитывать только тех, чьи книги получили достаточно отзывов (50 и более), чтобы результаты были статистически значимыми.

6. Изучить поведение активных пользователей — вычислить среднее количество обзоров у тех, кто поставил более 48 оценок, чтобы понять, насколько активная аудитория склонна оставлять текстовые отзывы.

7. Сделать выводы о предпочтениях читателей и активности пользователей, которые помогут определить направления развития продукта и его ценностное предложение.

## Имеющиеся данные

**Таблица books**  
Содержит данные о книгах:  
- book_id — идентификатор книги;  
- author_id — идентификатор автора;  
- title — название книги;  
- num_pages — количество страниц;  
- publication_date — дата публикации книги;  
- publisher_id — идентификатор издателя.

**Таблица authors**  
Содержит данные об авторах:  
- author_id — идентификатор автора;  
- author — имя автора.

**Таблица publishers**  
Содержит данные об издательствах:  
- publisher_id — идентификатор издательства;  
- publisher — название издательства.

**Таблица ratings**  
Содержит данные о пользовательских оценках книг:  
- rating_id — идентификатор оценки;  
- book_id — идентификатор книги;  
- username — имя пользователя, оставившего оценку;  
- rating — оценка книги.

**Таблица reviews**  
Содержит данные о пользовательских обзорах:  
- review_id — идентификатор обзора;  
- book_id — идентификатор книги;  
- username — имя автора обзора;  
- text — текст обзора.

## Исследование

### Ознакомление со структурой данных

In [1]:
# импортирую библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

In [2]:
# устанавливаю параметры
db_config = {'user': 'praktikum_student', # имя пользователя 
             'pwd': 'Sdf4$2;d-d30pp', # пароль 
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net', 
             'port': 6432, # порт подключения 
             'db': 'data-analyst-final-project-db'} # название базы данных 
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config) 

# сохраняю коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

query = '''SELECT * FROM books LIMIT 5'''

con=engine.connect() 

pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [3]:
query = '''SELECT * FROM authors LIMIT 5'''
con=engine.connect() 

pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [4]:
query = '''SELECT * FROM ratings LIMIT 5'''
con=engine.connect() 

pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [5]:
query = '''SELECT * FROM reviews LIMIT 5'''
con=engine.connect() 

pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [6]:
query = '''SELECT * FROM publishers LIMIT 5'''
con=engine.connect() 

pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


### Количество книг, выпущенных после 1 января 2000 года

In [7]:
query = """
SELECT COUNT (DISTINCT book_id) AS books_after_2000
FROM books
WHERE publication_date > '2000-01-01'
"""
df = pd.read_sql(query, engine)
display(df)

,books_after_2000
0,819


После 1 января 2000 года было выпущено 819 книг. Это более 80% всего датасета, что показывает, что купленный сервис чтения книг ориентировался по большей части на современную литературу (или перевыпуск классики, но это надо изучить отдельно).

### Количество обзоров на книгу и их средняя оценка

In [15]:
query = """
SELECT b.title,
    COUNT(DISTINCT rev.review_id) AS count_of_reviews,
    AVG(ra.rating) AS avg_rating
FROM books b
LEFT JOIN ratings ra ON b.book_id = ra.book_id
LEFT JOIN reviews rev ON b.book_id = rev.book_id
GROUP BY b.book_id, b.title
ORDER BY AVG(ra.rating) DESC, COUNT(DISTINCT rev.review_id) DESC
"""
df = pd.read_sql(query, engine)
display(df)

,title,count_of_reviews,avg_rating
0,A Dirty Job (Grim Reaper #1),4,5.00
1,School's Out—Forever (Maximum Ride #2),3,5.00
2,Moneyball: The Art of Winning an Unfair Game,3,5.00
3,Arrows of the Queen (Heralds of Valdemar #1),2,5.00
4,Wherever You Go There You Are: Mindfulness Me...,2,5.00
...,...,...,...
995,The World Is Flat: A Brief History of the Twen...,3,2.25
996,Drowning Ruth,3,2.00
997,His Excellency: George Washington,2,2.00
998,Junky,2,2.00


В датасете тысяча книг, на которые были написаны обзоры и которым были поставлены оценки.  
Наивысшую среднюю оценку — 5.0 — получили несколько книг, включая A Dirty Job и Moneyball, однако количество обзоров по ним невелико (2–4).
Самый низкий рейтинг - 1.50 - получила книга Harvesting the heart (на нее тоже было сделано всего 2 обзора).

### Издательство, выпустившее наибольшее количество книг объемом более 50 страниц

In [9]:
query = """
SELECT publisher
FROM publishers
WHERE publisher_id IN (SELECT publisher_id
                            FROM books
                            WHERE num_pages > 50
                            GROUP BY publisher_id
                            ORDER BY COUNT(DISTINCT book_id) DESC
                            LIMIT 1)
"""
df = pd.read_sql(query, engine)
display(df)

,publisher
0,Penguin Books


Лидером по числу полноформатных изданий стало Penguin Books. Что неудивительно, это одно из крупнейших международных издательств с долгой историей и масштабным каталогом.

### Автор с самой высокой средней оценкой книг

In [10]:
query = """
WITH rating_stats AS(
SELECT book_id,
    COUNT(rating_id) AS rating_count,
    AVG(rating) AS avg_rating
FROM ratings
GROUP BY book_id
HAVING COUNT(rating_id) >= 50
), 

author_stats AS(
SELECT a.author,
    rs.avg_rating
FROM rating_stats rs
JOIN books b ON rs.book_id=b.book_id
JOIN authors a ON b.author_id=a.author_id)

SELECT author
FROM author_stats
GROUP BY author
ORDER BY AVG(avg_rating) DESC
LIMIT 1
"""
df = pd.read_sql(query, engine)
display(df)

,author
0,J.K. Rowling/Mary GrandPré


Наивысшую среднюю оценку среди авторов получила J.K. Rowling (совместно с Mary GrandPré).  
Высокий рейтинг и популярность серии о Гарри Поттере достаточно ожидаемо обеспечили автору первое место.

## Среднее количество обзоров от активных пользователей

In [11]:
query = """
SELECT AVG(reviews_per_user) AS avg_review_count
FROM (SELECT username,
    COUNT(review_id) AS reviews_per_user
            FROM reviews
            WHERE username IN (SELECT username
                                FROM ratings
                                GROUP BY username
                                HAVING COUNT(rating_id) > 48)
            GROUP BY username) AS user_reviews
"""
df = pd.read_sql(query, engine)
display(df)

,avg_review_count
0,24.0


Активные пользователи (поставившие более 48 оценок) в среднем оставили 24 обзора.
Это может говорить о том, что наиболее вовлечённые участники платформы не только активно оценивают книги, но и часто делятся развёрнутыми впечатлениями.

## Вывод

Большая часть представленных в датасете изданий относится к последним десятилетиям — после 2000 года было выпущено 819 книг, что подчёркивает современную направленность базы сервиса.  
Оценки пользователей варьируются от 1.5 до 5.0, при этом высокая средняя оценка не всегда сопровождается большим числом отзывов, что указывает на необходимость учитывать популярность книги при интерпретации рейтингов.

Среди издательств лидирует Penguin Books, подтверждая репутацию одного из крупнейших игроков на мировом книжном рынке.  
Наивысшую среднюю оценку среди авторов получила J.K. Rowling, чьи произведения сохраняют устойчивую популярность и признание читателей.  
Анализ пользовательской активности показал, что наиболее вовлечённые читатели (с более чем 48 оценками) в среднем оставляют около 24 обзоров, что говорит о высокой готовности делиться своими впечатлениями и формировать контент сообщества.